# Using compression

> Objectives:
> * How to compress chunked datasets
> * Learn how to fine-tune the HDF5 compression pipeline to suit your needs
> * How to use pandas for reading CSV files

## Load movielens datasets

In [9]:
import os
import numpy as np
import pandas as pd
import tables

In [10]:
# Import CSV files via pandas
dset = 'movielens-1m'
fdata = os.path.join(dset, 'ratings.dat.gz')
fitem = os.path.join(dset, 'movies.dat.gz')

# pass in column names for each CSV
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(fdata, sep=';', names=r_cols)

m_cols = ['movie_id', 'title', 'genres']
movies = pd.read_csv(fitem, sep=';', names=m_cols,
                     dtype={'title': object, 'genres': object})

In [11]:
movies.ftypes

movie_id     int64:dense
title       object:dense
genres      object:dense
dtype: object

In [12]:
ratings.ftypes

user_id           int64:dense
movie_id          int64:dense
rating            int64:dense
unix_timestamp    int64:dense
dtype: object

## Storing in HDF5/PyTables in compressed form

In [13]:
import os
import shutil
data_dir = "compression"
if os.path.exists(data_dir):
    shutil.rmtree(data_dir)
os.mkdir(data_dir)

In [14]:
def to_hdf5(ratings, movies, filters):
    
    class Ratings(tables.IsDescription):
        user_id = tables.Int32Col(pos=0)
        movie_id = tables.Int32Col(pos=1)
        rating = tables.Int8Col(pos=2)
        unix_timestamp = tables.Int64Col(pos=3)
    
    class Movies(tables.IsDescription):
        movie_id = tables.Int32Col(pos=0)
        title = tables.StringCol(100, pos=1)
        genres = tables.StringCol(50, pos=2)
    
    def get_filename(filters):
        if filters.complevel != 0:
            complib = filters.complib if ":" not in filters.complib else filters.complib.replace(":", "-")
            shuffle = "shuffle" if filters.shuffle else "noshuffle"
            filename = "%s/%s-%d-%s.h5" % (data_dir, complib, filters.complevel, shuffle)
        else:
            filename = "%s/no-compressed.h5" % (data_dir,)
        return filename

    filename = get_filename(filters)
    print("Creating file:", filename)
    with tables.open_file(filename, "w") as f:
        table_ratings = f.create_table(f.root, "ratings", Ratings, filters=filters, expectedrows=len(ratings))
        table_ratings.append([ratings[col].values for col in ratings.ftypes.keys()])
        table_movies = f.create_table(f.root, "movies", Movies, filters=filters, expectedrows=len(movies))
        table_movies.append([movies[col].values for col in movies.ftypes.keys()])
    return filename

In [20]:
%%time
filters = tables.Filters(complevel=5, shuffle=True)
h5file = to_hdf5(ratings, movies, filters)

Creating file: compression/zlib-5-shuffle.h5
CPU times: user 381 ms, sys: 20.7 ms, total: 402 ms
Wall time: 403 ms


In [21]:
!ptdump -v {h5file}

/ (RootGroup) ''
/movies (Table(3883,), shuffle, zlib(5)) ''
  description := {
  "movie_id": Int32Col(shape=(), dflt=0, pos=0),
  "title": StringCol(itemsize=100, shape=(), dflt=b'', pos=1),
  "genres": StringCol(itemsize=50, shape=(), dflt=b'', pos=2)}
  byteorder := 'little'
  chunkshape := (425,)
/ratings (Table(1000209,), shuffle, zlib(5)) ''
  description := {
  "user_id": Int32Col(shape=(), dflt=0, pos=0),
  "movie_id": Int32Col(shape=(), dflt=0, pos=1),
  "rating": Int8Col(shape=(), dflt=0, pos=2),
  "unix_timestamp": Int64Col(shape=(), dflt=0, pos=3)}
  byteorder := 'little'
  chunkshape := (7710,)


### Exercise 1

PyTables comes with out-of-box support for a series of codecs.  Do a quick comparison between "zlib", "bzip2", and "blosc" for compression levels of 1 (fastest), 5 and 9 (slowest).  Which one compresses best?  Which one compresses faster?

Also, Blosc being a meta-compressor, it has support for different codecs internally that can be selected from PyTables in the "blosc:`codec`" form.  Do another comparison between internal Blosc codecs, namely, "blosc:blosclz" (the default), "blosc:lz4", "blosc:lz4hc", "blosc:snappy", "blosc:zlib" and "blosc:zstd".

Finally, avoid any compression totally (`complevel=0`).  How fast it is compared with existing codecs?

In [23]:
for complib in ("zlib", "bzip2", "blosc"):
    filters = tables.Filters(complevel=5, complib=complib, shuffle=True)
    %time to_hdf5(ratings, movies, filters)

Creating file: compression/zlib-5-shuffle.h5
CPU times: user 383 ms, sys: 23.8 ms, total: 407 ms
Wall time: 411 ms
Creating file: compression/bzip2-5-shuffle.h5
CPU times: user 2.37 s, sys: 19.2 ms, total: 2.39 s
Wall time: 2.4 s
Creating file: compression/blosc-5-shuffle.h5
CPU times: user 60.2 ms, sys: 12.8 ms, total: 73.1 ms
Wall time: 73.5 ms


In [25]:
%ls -lh {data_dir}

total 121712
-rw-r--r--  1 faltet  staff   5.0M May 18 10:37 blosc-5-shuffle.h5
-rw-r--r--  1 faltet  staff   5.0M May 18 10:38 blosc-blosclz-5-shuffle.h5
-rw-r--r--  1 faltet  staff   5.5M May 18 10:38 blosc-lz4-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.8M May 18 10:38 blosc-lz4hc-5-shuffle.h5
-rw-r--r--  1 faltet  staff   5.5M May 18 10:38 blosc-snappy-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.3M May 18 10:38 blosc-zlib-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.3M May 18 10:38 blosc-zstd-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.1M May 18 10:37 bzip2-5-shuffle.h5
-rw-r--r--  1 faltet  staff    17M May 18 10:34 no-compressed.h5
-rw-r--r--  1 faltet  staff   4.2M May 18 10:37 zlib-5-shuffle.h5


In [24]:
for complib in ("blosc:blosclz", "blosc:lz4", "blosc:lz4hc", "blosc:snappy", "blosc:zlib", "blosc:zstd"):
    filters = tables.Filters(complevel=5, complib=complib, shuffle=True)
    %time to_hdf5(ratings, movies, filters)

Creating file: compression/blosc-blosclz-5-shuffle.h5
CPU times: user 90.4 ms, sys: 17.8 ms, total: 108 ms
Wall time: 112 ms
Creating file: compression/blosc-lz4-5-shuffle.h5
CPU times: user 38.6 ms, sys: 13.1 ms, total: 51.7 ms
Wall time: 52.4 ms
Creating file: compression/blosc-lz4hc-5-shuffle.h5
CPU times: user 342 ms, sys: 15.6 ms, total: 358 ms
Wall time: 360 ms
Creating file: compression/blosc-snappy-5-shuffle.h5
CPU times: user 43.2 ms, sys: 12.3 ms, total: 55.5 ms
Wall time: 55.9 ms
Creating file: compression/blosc-zlib-5-shuffle.h5
CPU times: user 304 ms, sys: 14.9 ms, total: 319 ms
Wall time: 320 ms
Creating file: compression/blosc-zstd-5-shuffle.h5
CPU times: user 183 ms, sys: 14 ms, total: 197 ms
Wall time: 198 ms


In [26]:
%ls -lh {data_dir}

total 121712
-rw-r--r--  1 faltet  staff   5.0M May 18 10:37 blosc-5-shuffle.h5
-rw-r--r--  1 faltet  staff   5.0M May 18 10:38 blosc-blosclz-5-shuffle.h5
-rw-r--r--  1 faltet  staff   5.5M May 18 10:38 blosc-lz4-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.8M May 18 10:38 blosc-lz4hc-5-shuffle.h5
-rw-r--r--  1 faltet  staff   5.5M May 18 10:38 blosc-snappy-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.3M May 18 10:38 blosc-zlib-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.3M May 18 10:38 blosc-zstd-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.1M May 18 10:37 bzip2-5-shuffle.h5
-rw-r--r--  1 faltet  staff    17M May 18 10:34 no-compressed.h5
-rw-r--r--  1 faltet  staff   4.2M May 18 10:37 zlib-5-shuffle.h5


## Reading compressed datasets

In [27]:
files = list(os.walk(data_dir))[0][2]

In [28]:
files

['blosc-5-shuffle.h5',
 'blosc-blosclz-5-shuffle.h5',
 'blosc-lz4-5-shuffle.h5',
 'blosc-lz4hc-5-shuffle.h5',
 'blosc-snappy-5-shuffle.h5',
 'blosc-zlib-5-shuffle.h5',
 'blosc-zstd-5-shuffle.h5',
 'bzip2-5-shuffle.h5',
 'no-compressed.h5',
 'zlib-5-shuffle.h5']

In [29]:
for f in files:
    print("Reading file:", f)
    with tables.open_file(os.path.join(data_dir, f)) as h5f:
        %time h5f.root.ratings[:]

Reading file: blosc-5-shuffle.h5
CPU times: user 37.6 ms, sys: 13.7 ms, total: 51.3 ms
Wall time: 59.4 ms
Reading file: blosc-blosclz-5-shuffle.h5
CPU times: user 30.1 ms, sys: 9.1 ms, total: 39.2 ms
Wall time: 41.9 ms
Reading file: blosc-lz4-5-shuffle.h5
CPU times: user 21.5 ms, sys: 7.68 ms, total: 29.2 ms
Wall time: 30.3 ms
Reading file: blosc-lz4hc-5-shuffle.h5
CPU times: user 19.9 ms, sys: 7.07 ms, total: 27 ms
Wall time: 26.7 ms
Reading file: blosc-snappy-5-shuffle.h5
CPU times: user 29 ms, sys: 10.6 ms, total: 39.5 ms
Wall time: 49.2 ms
Reading file: blosc-zlib-5-shuffle.h5
CPU times: user 81.7 ms, sys: 7.45 ms, total: 89.1 ms
Wall time: 90.1 ms
Reading file: blosc-zstd-5-shuffle.h5
CPU times: user 35.7 ms, sys: 6.85 ms, total: 42.6 ms
Wall time: 43.1 ms
Reading file: bzip2-5-shuffle.h5
CPU times: user 664 ms, sys: 9.89 ms, total: 674 ms
Wall time: 677 ms
Reading file: no-compressed.h5
CPU times: user 7.56 ms, sys: 11.3 ms, total: 18.9 ms
Wall time: 18.9 ms
Reading file: zlib-5-

### Exercise 2

Which codec and compression level can read the fastest?  How it does compare with reading an uncompressed dataset?

### Exercise 3

Blosc can use multithreading for compressing/decompressing, although it is disabled by default.  You can enable a multithreaded Blosc in a series of ways, but perhaps the easiest is to set the "BLOSC_NTHREADS" environment variable to the desired number of threads (typically the available number of cores in your computer).

Execute the cell below and re-do the reading benchmarks and look at how the reading speed vary.  Pay special attention to the difference between the CPU times and wall times.

In [30]:
os.environ["BLOSC_NTHREADS"] = "4"  # set to any other number you prefer

In [32]:
for f in files:
    print("Reading file:", f)
    with tables.open_file(os.path.join(data_dir, f)) as h5f:
        %time h5f.root.ratings[:]

Reading file: blosc-5-shuffle.h5
CPU times: user 41.4 ms, sys: 16.6 ms, total: 58 ms
Wall time: 42 ms
Reading file: blosc-blosclz-5-shuffle.h5
CPU times: user 41.3 ms, sys: 23.4 ms, total: 64.6 ms
Wall time: 56.9 ms
Reading file: blosc-lz4-5-shuffle.h5
CPU times: user 25.8 ms, sys: 19 ms, total: 44.7 ms
Wall time: 32.8 ms
Reading file: blosc-lz4hc-5-shuffle.h5
CPU times: user 24.4 ms, sys: 18 ms, total: 42.3 ms
Wall time: 30 ms
Reading file: blosc-snappy-5-shuffle.h5
CPU times: user 31.2 ms, sys: 18.9 ms, total: 50.1 ms
Wall time: 34.1 ms
Reading file: blosc-zlib-5-shuffle.h5
CPU times: user 97.2 ms, sys: 26.8 ms, total: 124 ms
Wall time: 78.2 ms
Reading file: blosc-zstd-5-shuffle.h5
CPU times: user 45.2 ms, sys: 15.6 ms, total: 60.8 ms
Wall time: 39.4 ms
Reading file: bzip2-5-shuffle.h5
CPU times: user 709 ms, sys: 13.3 ms, total: 723 ms
Wall time: 732 ms
Reading file: no-compressed.h5
CPU times: user 6.93 ms, sys: 9.06 ms, total: 16 ms
Wall time: 15.7 ms
Reading file: zlib-5-shuffle.